<h5> Import the libraries </h5>

In [6]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup

<p> I define an array, URL and the HTML parser, </br>
    look for the table and iterated for each row and </br>
    value to append to the postal_codes array </p>
    
<p> After that I cast the array to a Data Frame Pandas, </br>
    Rename the columns, filtered for 'Not assigned' in Borough column </br>    
    Join the values through semi colon </br>
    Reset index data frame
    Update the Neighbourhood column for the Borough </p>

In [7]:
postal_codes_array = []
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.text, "html.parser")

postal_table = soup.find('table', {"class" : "wikitable sortable"})

for row in postal_table.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 3:
        postal_codes_array.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))

df = pd.DataFrame(postal_codes_array)
df.columns = ['PostalCode', 'Borough','Neighbourhood']
df = df[df.Borough != 'Not assigned']
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join)
df = df.reset_index()
df.loc[df['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = df['Borough']
df.shape
#df

(103, 3)

In [21]:
!wget -q -O 'canada_data.csv' https://cocl.us/Geospatial_data

df_coordinates = pd.read_csv("canada_data.csv") 
df_coordinates.columns = ['PostalCode', 'Latitude','Longitude']
df_coordinates = pd.merge(df, df_coordinates, on='PostalCode', how='inner')
#df_coordinates

In [19]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.4                |           py36_0         877 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         961 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The following packages will be UPDATED:

  conda                                        4.6.3-py36_0 --> 4.6.4-py36_0
  geopy              conda-forge/linux-64::geopy-1.11.0-py~ --> conda-forge/noarch::geopy-1.18.1-py_0



geopy-1.18.1         

<p> Import folium libraries </br>
    Define the geolocator object, search for Toronto latitude and longitude </br>
    Print Toronto Map </br>
    Then add the different Neighbourhoods that contain the word Toronto </br>
</p>

In [27]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
toronto_data = df_coordinates[df_coordinates['Borough'].str.contains('Toronto')]

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

The geograpical coordinate of Toronto City are 43.653963, -79.387207.
